In [542]:
import pandas

from datetime import datetime
# import zip



rateios_celesc = pandas.read_csv(r"C:\Users\Luan_Plin\Documents\COMPILADO CELESC\COMPILADO RATEIOS - CELESC.csv")

rateios_celesc.drop(["OBSERVAÇÃO"],axis=1, inplace=True)

rateios_celesc['PORCENTAGEM']=rateios_celesc['PORCENTAGEM'].apply(lambda x : float(x.replace(",",".").replace("%",""))/100)

rateios_celesc['BENEFICIÁRIA']=rateios_celesc['BENEFICIÁRIA'].apply(str)

rateios_celesc['GERADORA']=rateios_celesc['GERADORA'].apply(str)

rateios_celesc["datetime"]=rateios_celesc["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

injecao_geradores = pandas.read_csv(r"C:\Users\Luan_Plin\Documents\COMPILADO CELESC\Faturas PLIN - CELESC.csv")

injecao_geradores=injecao_geradores.rename({"ucGeradora":"GERADORA","referencia":"REFERÊNCIA"},axis=1)

columns_maintain = ["GERADORA", "REFERÊNCIA", "INJEÇÃO TP"]

injecao_geradores = injecao_geradores[columns_maintain]

injecao_geradores["INJEÇÃO TP"]=injecao_geradores["INJEÇÃO TP"].apply(lambda x : float(x)) 

injecao_geradores["GERADORA"]=injecao_geradores["GERADORA"].apply(str) 

injecao_geradores["datetime"]=injecao_geradores["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

leituras_celesc = pandas.read_csv(r"C:\Users\Luan_Plin\Documents\COMPILADO CELESC\LEITURAS__Geradorasdeenergia_CELESC_Conta Corrente_Ativo_Erro de leitura_Vencidas_05-08-2024.csv")

leituras_celesc=leituras_celesc.rename({"uc":"UC","date_ref":"REFERÊNCIA","kwh_compensado":"COMPENSADO","kwh_consumed":"CONSUMIDO"},axis=1)

leituras_celesc["COMPENSADO"]=leituras_celesc["COMPENSADO"].apply(lambda x : float(abs(float(x))))

columns_maintain=["UC","REFERÊNCIA","COMPENSADO","CONSUMIDO"]

# TRATAMENTO DE DADOS LEITURAS

leituras_celesc=leituras_celesc[columns_maintain]

leituras_celesc["REFERÊNCIA"]=leituras_celesc["REFERÊNCIA"].apply(lambda x: "01/"+x)

leituras_celesc["datetime"]=leituras_celesc["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

rateios_celesc=rateios_celesc.set_index(["GERADORA","datetime"])

injecao_geradores=injecao_geradores.set_index(["GERADORA","datetime"])

leituras_celesc.set_index(['UC',"datetime"],inplace=True)

# INSERÇÃO COLUNA INJEÇÃO CLIENTE USINA

def injecao_cliente_usina(row):
    try:
        inj_usina = injecao_geradores["INJEÇÃO TP"].loc[row.name].values
        inj_cliente_usina = inj_usina * row["PORCENTAGEM"]
        return float(format(inj_cliente_usina[0],".0f"))
    
    except:
        return None
    
rateios_celesc["INJ_CLIENTE_USINA"] = rateios_celesc.apply(injecao_cliente_usina,axis=1)

rateios_celesc.sort_index()

leituras_celesc.sort_index()

injecao_geradores.sort_index()

C:\Users\Luan_Plin\AppData\Local\Temp\ipykernel_16184\2703395471.py:60: PerformanceWarning: indexing past lexsort depth may impact performance.
  inj_usina = injecao_geradores["INJEÇÃO TP"].loc[row.name].values


REFERÊNCIA  INJEÇÃO TP
GERADORA datetime                          
18247410 2024-03-01  01/03/2024      3912.0
         2024-04-01  01/04/2024      2556.0
         2024-05-01  01/05/2024      3234.0
         2024-06-01  01/06/2024       796.0
43155997 2024-03-01  01/03/2024      6616.0
...                         ...         ...
58307858 2024-06-01  01/06/2024         0.0
58355356 2024-03-01  01/03/2024      7616.0
         2024-04-01  01/04/2024      4607.0
         2024-05-01  01/05/2024      6111.0
         2024-06-01  01/06/2024       617.0

[70 rows x 2 columns]

In [568]:
def dict_inj_usinas(row:pandas.Series):

    rateio_beneficiaria = rateios_celesc.query("datetime == {datetime}".format(datetime = row.name[1]))

    rateio_beneficiaria = rateio_beneficiaria.loc[rateio_beneficiaria["BENEFICIÁRIA"] == row.name[0]]



SyntaxError: invalid syntax (2785987759.py, line 7)

In [575]:
leituras_celesc.iloc[15].name[1]

Timestamp('2024-07-01 00:00:00')